In [15]:
#importing the required libraries
import numpy as np
import pandas as pd 
import glob
import csv 
#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords
import nltk


#vis
import pyLDAvis

import pyLDAvis.gensim_models


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
#Preparing the data 
##remove stopwords  
stopwords = stopwords.words("english")

In [3]:
print (stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [4]:
#in this step, I want to make sure my data loaded correctly.  
df = pd.read_csv(r"/Users/hajeralbalawi/Desktop/data/scholarfrt.csv") ["gs_rs"]

print (df[0][0:90])


… bias in the specific tasks of face detection, verification, identification, identity clu


In [5]:
data=df


In [6]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:90])

bias specific task face detection verification identification identity clustering gender p


In [7]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['bias', 'specific', 'task', 'face', 'detection', 'verification', 'identification', 'identity', 'clustering', 'gender', 'process', 'compliance', 'standard', 'just', 'as', 'important', 'result']


In [8]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1)]
as


In [22]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [21]:

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis


/Users/hajeralbalawi/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.245004  0.516514       1        1  41.142155
13     0.532464  0.070909       2        1  25.632048
10    -0.355484  0.273683       3        1   7.922781
20    -0.090486  0.329196       4        1   4.364387
12     0.187369 -0.324626       5        1   4.298358
5      0.307738 -0.111973       6        1   3.404448
3      0.120602  0.225975       7        1   3.187708
16    -0.160731 -0.293281       8        1   2.622094
14    -0.287871 -0.013542       9        1   2.380157
1      0.067318  0.066238      10        1   1.291101
17     0.173013  0.015920      11        1   1.231914
4     -0.130648  0.148291      12        1   1.076287
8     -0.159494 -0.109724      13        1   0.587164
29    -0.044531 -0.035209      14        1   0.159880
22    -0.018949 -0.076967      15        1   0.091001
21    -0.029054 -0.048018      16        1   0.074879
25    -0.025447 -0.045242      17        1   0.038681
26    -0.025447 -0.045242      18        1   0.038074
27    -0.025447 -0.045242      19        1   0.038074
24    -0.025447 -0.045242      20        1   0.038074
23    -0.025447 -0.045242      21        1   0.038074
28    -0.025447 -0.045242      22        1   0.038074
0     -0.025447 -0.045242      23        1   0.038074
19    -0.025447 -0.045242      24        1   0.038074
18    -0.025447 -0.045242      25        1   0.038074
11    -0.025447 -0.045242      26        1   0.038074
9     -0.025447 -0.045242      27        1   0.038074
7     -0.025447 -0.045242      28        1   0.038074
6     -0.025447 -0.045242      29        1   0.038074
15    -0.025447 -0.045242      30        1   0.038074, topic_info=           Term         Freq        Total Category  logprob  loglift
1          bias  1898.000000  1898.000000  Default  30.0000  30.0000
23      ethical   959.000000   959.000000  Default  29.0000  29.0000
38  recognition  1422.000000  1422.000000  Default  28.0000  28.0000
3    compliance   910.000000   910.000000  Default  27.0000  27.0000
37       facial  1368.000000  1368.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
24        legal     0.002232   160.491869  Topic30  -7.8018  -3.3099
26      monitor     0.002232    11.612001  Topic30  -7.8018  -0.6837
27     question     0.002232    46.757992  Topic30  -7.8018  -2.0766
28      receive     0.002232     7.152044  Topic30  -7.8018  -0.1990
29       regard     0.002232    46.962177  Topic30  -7.8018  -2.0810

[1525 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
80        5  0.865757        abuse
49        9  0.978947     accuracy
81        5  0.991022      address
69       13  0.869693       agency
32        2  0.988645  algorithmic
...     ...       ...          ...
58        9  0.949559       unfair
30        4  0.997175          use
133       4  0.980601      various
177       3  0.996428         well
107       7  0.975944         work

[113 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 14, 11, 21, 13, 6, 4, 17, 15, 2, 18, 5, 9, 30, 23, 22, 26, 27, 28, 25, 24, 29, 1, 20, 19, 12, 10, 8, 7, 16])